In [ ]:
!pip install -U sentence-transformers
!pip install pandas
!pip install accelerate==0.31.0
!pip install torch
!pip install summac
!pip install nltk
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 4.7 MB/s eta 0:00:00 0:00:01
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.0
    Uninstalling huggingface-hub-0.17.0:
      Successfully uninstalled huggingface-hub-0.17.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
summac 0.0.4 requires huggingface-hub<=0.17

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached huggingface_hub-0.17.0-py3-none-any.whl.metadata (13 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.50.2-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.50.1-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.50.0-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.48.2-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import pandas as pd

filepath = './articles_simplified_4.jsonl'
articles = pd.read_json(path_or_buf=filepath, lines=True)
texts = articles['text']
simplifieds = articles['simplified']
simplifieds2 = articles['simplified2']
simplifieds3 = articles['simplified3']

In [8]:
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ytzka14/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
from sentence_transformers import SentenceTransformer
import sklearn.metrics.pairwise as pairwise


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
csv_str = "article_id, sim1, sim2, sim3\n"

for i in range(articles.shape[0]):
  sentences = [texts[i], simplifieds[i], simplifieds2[i], simplifieds3[i]]
  embeddings = model.encode(sentences)
  similarities = pairwise.cosine_similarity(embeddings)
  sim1 = similarities[0][1]
  sim2 = similarities[0][2]
  sim3 = similarities[0][3]
  csv_str += f"{i}, {sim1}, {sim2}, {sim3}\n"

with open('all-MiniLM-L6-v2.csv', 'w') as f:
  f.write(csv_str)
  f.close()



/Users/ytzka14/github_projects/fvr_extension/others/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ytzka14/github_projects/fvr_extension/others/.conda/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/ytzka14/github_projects/fvr_extension/others/.conda/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/ytzka14/github_projects/fvr_extension/others/.conda/lib/python3.12/site-packages/transformers/utils/ge

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("biu-nlp/superpal")

model = AutoModelForSequenceClassification.from_pretrained("biu-nlp/superpal")
csv_str = "article_id, sim1, sim2, sim3\n"

for i in range(articles.shape[0]):
  sentences = [texts[i], simplifieds[i], simplifieds2[i], simplifieds3[i]]
  inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
  outputs = model(**inputs)
  logits = outputs.logits
  probs = logits.softmax(dim=1)
  print(probs)
  sim1 = probs[0][1].item()
  sim2 = probs[0][2].item()
  sim3 = probs[0][3].item()
  csv_str += f"{i}, {sim1}, {sim2}, {sim3}\n"

with open('superpal.csv', 'w') as f:
  f.write(csv_str)
  f.close()

IndexError: index 2 is out of bounds for dimension 0 with size 2

In [ ]:
# from summac.model_summac import SummaCZS, SummaCConv

# model_zs = SummaCZS(granularity='sentence', model_name='vitc', device='cpu')
# model_conv = SummaCConv(models=['vitc'], bins='percentile', granularity='sentence', nli_labels='e', device='cpu', start_file='default', agg='mean')

# zs_csv_str = "article_id, sim1, sim2, sim3\n"
# conv_csv_str = "article_id, sim1, sim2, sim3\n"

# for i in range(articles.shape[0]):
#   score_zs = model_zs.score([texts[i], texts[i], texts[i]], [simplifieds[i], simplifieds2[i], simplifieds3[i]])
#   score_conv = model_conv.score([texts[i], texts[i], texts[i]], [simplifieds[i], simplifieds2[i], simplifieds3[i]])

#   score_zs = score_zs["scores"]
#   score_conv = score_conv["scores"]
  
#   zs_csv_str += f"{i}, {score_zs[0]}, {score_zs[1]}, {score_zs[2]}\n"
#   conv_csv_str += f"{i}, {score_conv[0]}, {score_conv[1]}, {score_conv[2]}\n"

# with open('SummaCZS.csv', 'w') as f:
#   f.write(zs_csv_str)
#   f.close()

# with open('SummaCConv.csv', 'w') as f:
#   f.write(conv_csv_str)
#   f.close()

<All keys matched successfully>


KeyboardInterrupt: 